### Wrangle Exercises

This exercises uses the ```case.csv```, ```dept.csv```, and ```source.csv``` files from the San Antonio 311 call dataset.

#### Data Acquisition

**1. Read the case, department, and source data into their own spark dataframes.**

In [1]:
import pandas as pd
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.functions import to_date

#create the spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
# read each csv into its own spark dataframe
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

**2. Let's see how writing to the local disk works in spark:**

- Write the code necessary to store the source data in both csv and json format, store these as ```sources_csv``` and ```sources_json```
- Inspect your folder structure. What do you notice?

<font color = 'blue'> A separate directory was generated to store the json and csv files.

In [3]:
# write to json
source.write.json("source_json", mode="overwrite")

# write to csv
source.write.csv("source_csv", mode="overwrite")

**3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.**

In [4]:
source.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [5]:
#look at schema to inspect data types
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [6]:
#cast source_id as an int
source = source.withColumn('source_id', col('source_id').cast('int'))

source.printSchema()

root
 |-- source_id: integer (nullable = true)
 |-- source_username: string (nullable = true)



#### Data Wrangle

In [7]:
# view list of cases
case.show(1,vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [8]:
#Rename column for reader clarity
case = case.withColumnRenamed('SLA_due_date', 'case_due_date')

# Change to appropriate data types
case = (
    case.withColumn('case_late', col('case_late') =='YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)

case = case.withColumn('council_district',format_string('%04d', col('council_district')))
case = (
    case.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)



# # Cleanup text data
# df = df.withColumn('request_address', lower(trim(col('request_address'))))
# # Extract zipcode
# df = df.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

# # Create a `case_lifetime` feature
# df = (
#     df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
#     .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
#     .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
#     .drop('case_age', 'days_to_closed')
# )

# # Join departments and sources
# depts = spark.read.csv('data/dept.csv', header=True, inferSchema=True)
# sources = spark.read.csv('data/source.csv', header=True, inferSchema=True)

# df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')

# # # Train Test Split
# # train, test = df.randomSplit([.8, .2], seed=123)
# # train, validate, test = df.randomSplit([.7, .15, .15], seed=123)

**1. How old is the latest (in terms of days past SLA) currently open issue?**

In [9]:
#To query our dataframe with Spark SQL:
case.createOrReplaceTempView('case')

In [12]:
spark.sql('''
    SELECT datediff(current_timestamp, case_due_date) 
    AS days_past_due
    FROM case
    WHERE NOT case_closed
    ORDER BY days_past_due DESC
    LIMIT 5
    '''
         ).show()



+-------------+
|days_past_due|
+-------------+
|         1581|
|         1581|
|         1581|
|         1580|
|         1578|
+-------------+



In [13]:
case.where(case.case_status == 'Open').sort(asc('case_opened_date')).show(1, vertical = True)


-RECORD 0------------------------------------
 case_id              | 1013225646           
 case_opened_date     | 2017-01-01 13:48:00  
 case_closed_date     | null                 
 case_due_date        | 2017-01-17 08:30:00  
 case_late            | true                 
 num_days_late        | 348.6458333          
 case_closed          | false                
 dept_division        | Code Enforcement     
 service_request_type | No Address Posted    
 SLA_days             | 15.77859954          
 case_status          | Open                 
 source_id            | svcCRMSS             
 request_address      | 7299  SHADOW RIDG... 
 council_district     | 0006                 
only showing top 1 row



In [14]:
case.groupby('SLA_days').count().sort(desc('SLA_days')).show(1)

+--------+-----+
|SLA_days|count|
+--------+-----+
|  1420.0|    1|
+--------+-----+
only showing top 1 row



**How long has the oldest (in terms of days since opened) currently opened issue been open?**

In [15]:
case.withColumn("age_in_days", datediff(current_timestamp(), "case_opened_date")
).show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 0005                 
 age_in_days          | 1232                 
only showing top 1 row



**2.  How many Stray Animal cases are there?**

In [16]:
# show a count of cases where the service request type is stray animal
case.where(case.service_request_type == 'Stray Animal').count()

26760

**3. How many service requests that are assigned to the Field Operations department (```dept_division```) are not classified as "Officer Standby" request type (```service_request_type```)?**

In [20]:
#filter first by service requests in field operations, then filter to include only those whose
#request type is not equal to officer standby -- then count them

(case.filter(case.dept_division == 'Field Operations')
        .filter(case.service_request_type != 'Officer Standby')
        .count()
)


113902

**4. Convert the ```council_district``` column to a string column.**

In [21]:
# format column to string datatypes
case = case.withColumn("council_district",col("council_district").cast("string"))

case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)



**5. Extract the year from the ```case_closed_date``` column.**

In [22]:
#select the year from the column, create a new column for the year
case.select('case_closed_date', year('case_closed_date')).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



**6. Convert ```num_days_late``` from days to hours in new columns ```num_hours_late```.**

In [23]:
# use withColumn to create a new column "num_hours_late" and convert value by multiplying days by 24

(case.withColumn('num_hours_late', case.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

**7. Join the case data with the source and department data.**

In [25]:
# Join departments and sources
depts = spark.read.csv('dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('source.csv', header=True, inferSchema=True)

case = case.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')

In [26]:
case.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



**8. Are there any cases that do not have a request source?**

In [28]:
(
    case.select(case.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [30]:
#Another way:
case.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



**9. What are the top 10 service request types in terms of number of requests?**

In [32]:
(case.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False))



+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



**10. What are the top 10 service request types in terms of average days late?**

In [33]:
# just where case_late is true
(case.where('case_late') 
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False))

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



**11. Does number of days late depend on department?**

In [34]:

(case.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False))

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



In [36]:
case.groupby('dept_name').count().show(truncate=False)

+-------------------------+------+
|dept_name                |count |
+-------------------------+------+
|Animal Care Services     |119362|
|null                     |198   |
|Solid Waste Management   |286287|
|Development Services     |1397  |
|Trans & Cap Improvements |97841 |
|Customer Service         |2889  |
|Metro Health             |5313  |
|Parks and Recreation     |19964 |
|Code Enforcement Services|321984|
|City Council             |34    |
+-------------------------+------+



**12. How do number of days late depend on department and request type?**

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [37]:
(case.filter("case_closed")
#     .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(asc("days_late"))
    .show(40, truncate=False))

+------------------------+--------------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                        |days_late|n_cases|
+------------------------+--------------------------------------------+---------+-------+
|City Council            |Request for Research/Information            |null     |5      |
|Trans & Cap Improvements|Engineering Design                          |-1413.8  |1      |
|Trans & Cap Improvements|Signal Timing Modification By Engineer      |-1352.0  |22     |
|Animal Care Services    |Stray Animal                                |-998.8   |27346  |
|Parks & Recreation      |Major Park Improvement Install              |-278.3   |271    |
|Trans & Cap Improvements|Sidewalk Cost Sharing Program               |-177.8   |131    |
|DSD/Code Enforcement    |Multi Tenant Exterior                       |-135.7   |84     |
|DSD/Code Enforcement    |CPS Energy Towers                           |-129.8   |511    |
|DSD/Code 

In [38]:
(case.filter(col("service_request_type") == "Stray Animal")
    .select("case_opened_date", "case_due_date", "case_closed_date")
    .withColumn("days_to_resolve", datediff("case_due_date", "case_opened_date"))
    .show())

+-------------------+-------------------+-------------------+---------------+
|   case_opened_date|      case_due_date|   case_closed_date|days_to_resolve|
+-------------------+-------------------+-------------------+---------------+
|2018-01-01 00:42:00|2020-09-26 00:42:00|2018-01-01 12:29:00|            999|
|2018-01-01 08:39:00|2020-09-26 08:39:00|2018-01-01 12:30:00|            999|
|2018-01-01 09:51:00|2020-09-26 09:51:00|2018-01-01 11:53:00|            999|
|2018-01-01 10:39:00|2020-09-26 10:39:00|2018-01-01 12:29:00|            999|
|2018-01-01 10:44:00|2020-09-26 10:44:00|2018-01-01 12:43:00|            999|
|2018-01-01 10:52:00|2020-09-26 10:52:00|2018-01-01 11:53:00|            999|
|2018-01-01 11:11:00|2020-09-26 11:11:00|2018-01-01 11:53:00|            999|
|2018-01-01 11:46:00|2020-09-26 11:46:00|2018-01-01 12:26:00|            999|
|2018-01-01 11:58:00|2020-09-26 11:58:00|2018-01-01 12:26:00|            999|
|2018-01-01 12:28:00|2020-09-26 12:28:00|2018-01-01 12:44:00|   

## NOTES FROM WALKTHROUGH

## Handling Dates

Quick Recap: getting data from spark dataframes:

- `.show(n)`: prints the first `n` rows. Doesn't produce a value that can be used later
- `.first`: gives us the first row object
- `.head(n)`: gives us a list of the first `n` row objects
- `.collect()`: turns *all* the rows into a list of row objects **be careful here**

In [41]:
case.select(max('case_opened_date'), max("case_closed_date")).collect()

[Row(max(case_opened_date)=datetime.datetime(2018, 8, 8, 10, 38), max(case_closed_date)=datetime.datetime(2018, 8, 8, 10, 38))]

In [42]:
max_date = case.select(max('case_opened_date'), max('case_closed_date')).first()[0]
max_date

datetime.datetime(2018, 8, 8, 10, 38)

In [43]:
max_date = max_date.strftime('%Y-%m-%d %H:%M:%S')
max_date

'2018-08-08 10:38:00'

In [46]:
case = (case.withColumn('case_age', datediff(lit(max_date), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed'))

In [47]:
case

DataFrame[source_id: string, dept_division: string, case_id: int, case_opened_date: timestamp, case_closed_date: timestamp, case_due_date: timestamp, case_late: boolean, num_days_late: double, case_closed: boolean, service_request_type: string, SLA_days: double, case_status: string, request_address: string, council_district: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string, source_username: string, case_lifetime: int]

## Sidebar: Python Code Formatting

Indentation conveys nesting

autoformatting tools remove the discussion

In [48]:
case = (case.withColumn('case_age', datediff(lit(max_date), 'case_opened_date')).withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date')).withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age'))).drop('case_age', 'days_to_closed'))

In [ ]:
df = (
    df.withColumn("case_age", datediff(lit(max_date), "case_opened_date"))
    .withColumn("days_to_closed", datediff("case_closed_date", "case_opened_date"))
    .withColumn(
        "case_lifetime",
        when(col("case_closed"), col("days_to_closed")).otherwise(col("case_age")),
    )
    .drop("case_age", "days_to_closed")
)

In [ ]:
(df.withColumn('case_age', datediff(lit(max_date), 'case_opened_date'))
 .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
 .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
 .drop('case_age', 'days_to_closed'))

1. Install black -- the python code formatter

    ```
    python -m pip install black
    ```
    
1. Create an alias

    ```
    alias fmt-clipboard='pbpaste | black -q - | pbcopy'
    ```

    Put this line in your `.zshrc` or `.bash_profile`
    
    ```
    echo $0
    ```
    
    Will tell you what shell you are using
    
1. Open a new terminal

1. Copy the code you want to format

1. Run the `fmt-clipboard` command

1. Paste the formatted code


To format a `.py` file

```
black script.py
```

`black-nb`